In [ ]:
from __future__ import print_function

import os
from os import path
import sys


import pandas as pd
import tensorflow as tf
import numpy as np
import cv2
import matplotlib.pyplot as plt
import scipy.misc
from tensorflow.keras.optimizers import Adam
import SimpleITK as sitk
from scipy import ndimage, misc
import glob

import json
import torch
import torch.nn as nn
from torch.nn import init
import time

import imutils

devices = tf.config.experimental.list_physical_devices('GPU')
if devices:
    # If GPUs are available, enable memory growth on them
    for device in devices:
        tf.config.experimental.set_memory_growth(device, True)
    # Set the first GPU as visible
    tf.config.experimental.set_visible_devices(devices[0], 'GPU')
else:
    # If no GPUs are available, list all CPUs
    devices = tf.config.experimental.list_physical_devices('CPU')
    # Set the first CPU as visible (typically there's only one)
    tf.config.experimental.set_visible_devices(devices[0], 'CPU')


sys.path.append('model/ProGNet/')
from ProGNet_model import *

import sys

sys.path.append('model/HistSeg/')
from metric import dice_coef_multilabel
from dilated_unet import Segmentation_model
from dataset import ImageProcessor, DataGenerator
from utils import soft_to_hard_pred, keep_largest_connected_components
from hist_to_seg import *

sys.path.append('model/GcGAN/')
from models.gc_cycle_gan_model import *


sys.path.append('./util')
from slice_corre import *
from  determine_angle_flip import *


sys.path.append('model/RegNet/network/')
from model import *
from image_reader import *
from loss import *
from transform import *
from combine_transforms import *
from tps import *


netG_AB = networks.define_G(input_nc=3, output_nc=3, ngf=64, which_model_netG='resnet_6blocks', norm='instance', use_dropout=False, init_type='xavier')
netG_AB.load_state_dict(torch.load('pretrained_models/MRI_Hist_gc_gan_128x128_180_net_G_AB.pth'))
netG_AB.eval()

sys.path.append('model/CycleGAN/')
from models_ink.pix2pix_model import *

model_ink = networks_ink.define_G(input_nc=3, output_nc=3, ngf=64, netG='resnet_6blocks', norm='batch', use_dropout=True, init_type='normal')
model_ink.load_state_dict(torch.load('pretrained_models/hist_ink_pix2pix_1024x1024_latest_net_G.pth'))
model_ink.eval()


model_aff  = tf.keras.models.load_model('pretrained_models/GcGAN_Pseudo_Multimodal_vgg16_affine_epoch_15.h5')
model_tps  = tf.keras.models.load_model('pretrained_models/GcGAN_Pseudo_Multimodal_vgg16_tps_epoch_15.h5')
model_aff_slice_corres  = tf.keras.models.load_model('pretrained_models/GcGAN_Pseudo_Multimodal_vgg16_affine_64x64_epoch_15.h5')
model_tps_slice_corres  = tf.keras.models.load_model('pretrained_models/GcGAN_Pseudo_Multimodal_vgg16_tps_64x64_epoch_15.h5')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
unet_model = Segmentation_model(filters=32,
                                    in_channels=3,
                                    n_block=4,
                                    n_class=2)
# load model weights
unet_model.load_state_dict(torch.load('pretrained_models/drunet_histo.pt', map_location=device))
# move to GPU
unet_model.to(device)

# model evaluation mode
unet_model.eval()

In [ ]:
# use predefined angle and slice correspondences, only for advanced users
# use_angle_flipping = True
# use_slice_correspondence = True
# use_t2_mask = True

use_angle_flipping = False
use_slice_correspondence = False
use_t2_mask = False

subjects = ["aaa0043"]
gd = [9]

reg_path = "registration_jsons/"

mri_path = 'dataset/MRI/' 

### AI + Human Angle and Flipping Prediction
if use_angle_flipping:
    data = pd.read_csv('./rotation_flipping_manual.csv')
    image_names = list(data.iloc[:,0])
    AI_human_angles = np.array(data.iloc[:,1])
    AI_human_flipping = np.array(data.iloc[:,2])


standardHist = 'model/ProGNet/std_hist_T2.npy'
ProGNetModelPath = 'pretrained_models/prognet_t2.h5'



slice_corr_times = np.zeros(len(subjects))

slice_corr_pred = np.zeros(len(gd))


image_size_hr = 1024
image_size = 128
image_size_slice_cor = 64

out_path = './results/'

factor = 0.15
scaling = 0.0001
#scaling = 0.0000

### load trained models
model_angle = tf.keras.models.load_model('pretrained_models/vgg16_angle_prediction_iteration_6000.h5')
model_angle_high = tf.keras.models.load_model('pretrained_models/vgg16_224x224_vgg16_angle_prediction_iteration_500.h5')




In [ ]:
index = 0

for index_ in range(len(subjects)):
    start = time() 
    sub = subjects[index_]
    print(sub)
    out_dir = out_path + sub 
    if not os.path.isdir(out_path):
        os.mkdir(out_path)
    if not os.path.isdir(out_dir):
        os.mkdir(out_dir)
    
    ### parse registraiton json
    reg_json = json.load(open(reg_path + sub + '.json'),)
    hist_json = json.load(open(reg_json["moving"]),)
    raw_mri_path = reg_json["fixed"]
    
    ### segmentation prostate on mri
    if use_t2_mask:
        t2_mask = sitk.ReadImage('dataset/MRI/' + sub + '_prostate_label.nii.gz')
    else:     
        ProGNet(raw_mri_path, out_dir, standardHist, ProGNetModelPath)
        t2_mask = sitk.ReadImage(out_dir + '/predictions/' + raw_mri_path.split('/')[-1].split('.')[0] + '_prostate_label.nii')>0
    
    slices = list(hist_json.keys())
    num_of_slices = len(slices)
    
    #### read T2-w MRI
    t2_ref =  sitk.ReadImage(raw_mri_path)
    normalized_mri_path = out_dir + '/resample_normalize/' + raw_mri_path.split('/')[-1].split('.')[0] + '_res_hm.nii'
    
    t2_vol = sitk.ReadImage(normalized_mri_path)    
    
    ### resample normalized mri and prostate mask to original mri space
    res = sitk.ResampleImageFilter()
    res.SetInterpolator(sitk.sitkNearestNeighbor)
    res.SetReferenceImage(t2_ref)
    t2_vol = res.Execute(t2_vol)
    t2_mask = res.Execute(t2_mask)
    
    spacing_mri = t2_vol.GetSpacing()[0]
    
    threshold = 400/(spacing_mri*spacing_mri)
    
    ### convert image to array
    t2_vol_array = sitk.GetArrayFromImage(t2_vol)
    maxV = np.max(t2_vol_array)
    minV = np.min(t2_vol_array)
    
    t2_vol_array = 255*(t2_vol_array - minV)/(maxV - minV)
    
    
    t2_mask_array = sitk.GetArrayFromImage(t2_mask)
    #### remove outliers in t2_mask
    for i in range(t2_mask_array.shape[0]):
        if (np.sum(t2_mask_array[i]) < threshold):
            t2_mask_array[i] = 0
    ####
    t2_vol_masked_array = t2_vol_array*t2_mask_array
    
    
    ### create array to store histology volume, histology segmentation, angle, and flipping
    angles = np.zeros(num_of_slices)
    flips = np.zeros(num_of_slices)
    hist_vol_array = np.zeros((num_of_slices,image_size_hr,image_size_hr,3))
    hist_seg_array = np.zeros((num_of_slices,image_size_hr,image_size_hr,3))
    hist_urethra_array = np.zeros((num_of_slices,image_size_hr,image_size_hr,3))
    hist_lmk_array = np.zeros((num_of_slices,image_size_hr,image_size_hr,3))
    hist_cancer_array = np.zeros((num_of_slices,image_size_hr,image_size_hr,3))
    ###
    
    ### get histopathology images and labels from json file
    ### segmentation prostate gland on MRI and histopathology by deep learning methods
    ### estimate rotation and flipping for each histopathology
    for s in range(num_of_slices):
        img_path = hist_json[slices[s]]['filename']
        img_fr = cv2.imread(img_path)
        
        img_cancer = np.zeros(img_fr.shape)
        
        regions = hist_json[slices[s]]['regions']
        
        if 'region01' in regions:
            region01_path = regions['region01']['filename']
            region01 = cv2.imread(region01_path)
            img_cancer[region01>0] = 1
        
        if 'region09' in regions:
            urethra_path = regions['region09']['filename']
            img_urethra = cv2.imread(urethra_path)
        else:
            img_urethra = np.zeros(img_cancer.shape)
        
        if 'region10' in regions:
            lmk_path = regions['region10']['filename']
            img_lmk = cv2.imread(lmk_path) 
        else:
            img_lmk = np.zeros(img_cancer.shape)
        
        if use_angle_flipping:
            index_temp = image_names.index(img_path)
            angle = AI_human_angles[index_temp]
            flip = AI_human_flipping[index_temp]
        else:
            angle, flip = angle_flip_pred_AI_ink(model_angle, model_ink, img_fr,img_size = 64, ink_size = 512, pad_ratio = 1.1185)
        angles[s] = angle
        flips[s] = flip
        
        print(img_path)
        print(angle)
        print(flip)
#         angle = 0
#         flip = 0
        
        ### segment prostate gland from histopathology slice
        hist_seg = hist_to_seg(unet_model,img_fr)
        hist_seg[hist_seg<254] = 0
        
        w_ = img_fr.shape[0]
        h_ = img_fr.shape[1]
        hist_pad_size = int(np.maximum(w_,h_)*2)
        
        hist_pad = np.ones((hist_pad_size, hist_pad_size, 3))*244
        hist_pad = hist_pad.astype('float32')
        mask_pad = np.zeros((hist_pad_size, hist_pad_size, 3))
        mask_pad = mask_pad.astype('float32')
        urethra_pad = np.zeros((hist_pad_size, hist_pad_size, 3))
        urethra_pad = urethra_pad.astype('float32')
        lmk_pad = np.zeros((hist_pad_size, hist_pad_size, 3))
        lmk_pad = lmk_pad.astype('float32')
        cancer_pad = np.zeros((hist_pad_size, hist_pad_size, 3))
        cancer_pad = cancer_pad.astype('float32')
        
        x_offset = int((hist_pad_size - w_)/2)
        y_offset = int((hist_pad_size - h_)/2)
        hist_pad[x_offset:x_offset+w_,y_offset:y_offset+h_,:] = img_fr
        mask_pad[x_offset:x_offset+w_,y_offset:y_offset+h_,:] = hist_seg
        urethra_pad[x_offset:x_offset+w_,y_offset:y_offset+h_,:] = img_urethra
        lmk_pad[x_offset:x_offset+w_,y_offset:y_offset+h_,:] = img_lmk
        cancer_pad[x_offset:x_offset+w_,y_offset:y_offset+h_,:] = img_cancer

        
        points = np.argwhere(mask_pad[:,:,0] > 0)
        points = np.fliplr(points)
        y, x, h, w = cv2.boundingRect(points)

        if w>=h:
            x_offset = int(w*factor)
            y_offset = int((w - h + 2*x_offset)/2)
        else:
            y_offset = int(h*factor)
            x_offset = int((h - w + 2*y_offset)/2)
        
        hist_pad_crop = hist_pad[x - x_offset:x+w+x_offset, y - y_offset:y+h+y_offset,:]
        mask_pad_crop = mask_pad[x - x_offset:x+w+x_offset, y - y_offset:y+h+y_offset,:]
        urethra_pad_crop = urethra_pad[x - x_offset:x+w+x_offset, y - y_offset:y+h+y_offset,:]
        lmk_pad_crop = lmk_pad[x - x_offset:x+w+x_offset, y - y_offset:y+h+y_offset,:]
        cancer_pad_crop = cancer_pad[x - x_offset:x+w+x_offset, y - y_offset:y+h+y_offset,:]
        
        hist_pad_crop = cv2.resize(hist_pad_crop,(image_size_hr,image_size_hr))
        mask_pad_crop = cv2.resize(mask_pad_crop,(image_size_hr,image_size_hr))
        urethra_pad_crop = cv2.resize(urethra_pad_crop,(image_size_hr,image_size_hr))
        lmk_pad_crop = cv2.resize(lmk_pad_crop,(image_size_hr,image_size_hr))
        cancer_pad_crop = cv2.resize(cancer_pad_crop,(image_size_hr,image_size_hr))
        
        if flip == 1:
            hist_pad_crop = cv2.flip(hist_pad_crop,1)
            mask_pad_crop = cv2.flip(mask_pad_crop,1)
            urethra_pad_crop = cv2.flip(urethra_pad_crop,1)
            lmk_pad_crop = cv2.flip(lmk_pad_crop,1)
            cancer_pad_crop = cv2.flip(cancer_pad_crop,1)
        
        hist_pad_crop = imutils.rotate(hist_pad_crop, int(-angle))
        hist_pad_crop[hist_pad_crop<10] = 244
        
        mask_pad_crop = imutils.rotate(mask_pad_crop, int(-angle))
        urethra_pad_crop = imutils.rotate(urethra_pad_crop, int(-angle))
        lmk_pad_crop = imutils.rotate(lmk_pad_crop, int(-angle))
        cancer_pad_crop = imutils.rotate(cancer_pad_crop, int(-angle))
        
        hist_vol_array[s] = hist_pad_crop
        hist_seg_array[s] = mask_pad_crop
        hist_urethra_array[s] = urethra_pad_crop
        hist_lmk_array[s] = lmk_pad_crop
        hist_cancer_array[s] = cancer_pad_crop  
    
    if use_slice_correspondence:
        ind = gd[index]
    else: 
        ind = get_slice_correspondence(model_aff_slice_corres, model_tps_slice_corres, netG_AB, hist_vol_array*hist_seg_array/255.0, t2_vol_masked_array, image_size_slice_cor, scaling, factor)
    end = time()
    
    slice_corr_times[index_] = end - start
    
    slice_corr_pred[index_] = ind
    
    print("Running Time is: ", end - start)
    
    print("First MRI slice is: ", ind)
    print("---------------------------")
    print("Ground truth is: ", gd[index])

    print("Error in slice correspondence prediction is: ", np.abs(ind - gd[index]))
    
    
    
#     #### remove slices that do not belong to the prostate mask
    for i in range(t2_vol_masked_array.shape[0]):
        if i<ind or i>= ind + num_of_slices:
            t2_vol_masked_array[i] = 0
            t2_mask_array[i] = 0
            
            
        ##### register MRI with histopathology
    for s in range(num_of_slices):
        ### get histopathology slices
        hist_vol_pad_crop = hist_vol_array[s]
        hist_mask_pad_crop = hist_seg_array[s]
        hist_urethra_pad_crop = hist_urethra_array[s] 
        hist_landmark_pad_crop = hist_lmk_array[s]
        hist_cancer_pad_crop = hist_cancer_array[s]

        hist_vol_pad_crop = hist_vol_pad_crop.astype('float32')
        hist_mask_pad_crop = hist_mask_pad_crop.astype('float32')
        hist_urethra_pad_crop = hist_urethra_pad_crop.astype('float32')
        hist_cancer_pad_crop = hist_cancer_pad_crop.astype('float32')
        hist_landmark_pad_crop = hist_landmark_pad_crop.astype('float32')

        #### crop mri
        mri_vol_slice = np.zeros((t2_vol_masked_array.shape[1], t2_vol_masked_array.shape[2],3))
        mri_vol_slice[:,:,0] = t2_vol_masked_array[s+ind]
        mri_vol_slice[:,:,1] = t2_vol_masked_array[s+ind]
        mri_vol_slice[:,:,2] = t2_vol_masked_array[s+ind]

        mri_vol_slice = mri_vol_slice*25.5/(np.max(mri_vol_slice)/10)
        mri_vol_slice = mri_vol_slice.astype('uint8')

        points = np.argwhere(t2_vol_masked_array[s+ind] > 0)
        points = np.fliplr(points)
        y_m, x_m, h_m, w_m = cv2.boundingRect(points)

        if w_m>=h_m:
            x_offset_m = int(w_m*factor)
            y_offset_m = int((w_m - h_m + 2*x_offset_m)/2)
        else:
            y_offset_m = int(h_m*factor)
            x_offset_m = int((h_m - w_m + 2*y_offset_m)/2)


        mri_vol_crop = mri_vol_slice[x_m - x_offset_m:x_m+w_m+x_offset_m, y_m - y_offset_m:y_m+h_m+y_offset_m,:]


        m = tf.constant([1.0,0,0,0,1.0,0])
        m = tf.expand_dims(m,axis=0)

        if s==0:
            spacing_hist = spacing_mri*mri_vol_crop.shape[0]/hist_vol_pad_crop.shape[1]

        mri_input = cv2.resize(mri_vol_crop,(image_size,image_size))
        mri_input = np.expand_dims(mri_input,axis = 0)
        mri_input = mri_input.astype('float32')

        hist_input = cv2.resize(hist_vol_pad_crop*hist_mask_pad_crop/255.0,(image_size,image_size))
        hist_input = np.expand_dims(hist_input,axis = 0)
        hist_input = hist_input.astype('float32')
        
        
        mri_mask_batch = tf.convert_to_tensor(np.where(mri_input > 0, 255.0, 0), dtype=tf.float32)
        hist_mask_batch = tf.convert_to_tensor(np.where(hist_input > 0, 255.0, 0), dtype=tf.float32)


        theta = model_aff(([mri_mask_batch, hist_mask_batch]))
        #theta = model_aff(([mri_input,hist_input]))
        
        
        
        theta = tf.math.scalar_mul(scaling ,theta ) + m
        theta = np.expand_dims(theta,axis = 0)

        hist_deformed = affine_transformer_network(hist_input,theta)

        theta_tps = model_tps(([mri_input,hist_deformed]))
        theta_tps = tf.math.scalar_mul(scaling ,theta_tps )
        theta_tps = np.expand_dims(theta_tps,axis = 0)



        ### apply theta to deform histology images
        hist_vol_crop = np.squeeze(ThinPlateSpline(affine_transformer_network(np.expand_dims(hist_vol_pad_crop,axis=0), theta),theta_tps).numpy(),axis=0)
        hist_vol_crop[hist_vol_crop.astype(int)<30] = 244
        hist_mask_crop = np.squeeze(ThinPlateSpline(affine_transformer_network(np.expand_dims(hist_mask_pad_crop,axis=0), theta),theta_tps).numpy(),axis=0)
        hist_urethra_crop = np.squeeze(ThinPlateSpline(affine_transformer_network(np.expand_dims(hist_urethra_pad_crop,axis=0), theta),theta_tps).numpy(),axis=0)
        hist_cancer_crop = np.squeeze(ThinPlateSpline(affine_transformer_network(np.expand_dims(hist_cancer_pad_crop,axis=0), theta),theta_tps).numpy(),axis=0)
        hist_landmark_crop = np.squeeze(ThinPlateSpline(affine_transformer_network(np.expand_dims(hist_landmark_pad_crop,axis=0), theta),theta_tps).numpy(),axis=0)

        resize0 = int(spacing_mri*mri_vol_crop.shape[0]/(spacing_hist))
        resize1 = int(spacing_mri*mri_vol_crop.shape[1]/(spacing_hist))
        hist_vol_crop = cv2.resize(hist_vol_crop,(resize0,resize1))
        hist_mask_crop = cv2.resize(hist_mask_crop,(resize0,resize1))
        hist_cancer_crop = cv2.resize(hist_cancer_crop,(resize0,resize1))
        hist_urethra_crop = cv2.resize(hist_urethra_crop,(resize0,resize1))
        hist_landmark_crop = cv2.resize(hist_landmark_crop,(resize0,resize1))


        if s==0:
            hist_vol_3D = np.ones((num_of_slices, 3*hist_vol_crop.shape[0], 3*hist_vol_crop.shape[1], 3))*244
            hist_mask_3D = np.zeros((num_of_slices, 3*hist_mask_crop.shape[0], 3*hist_mask_crop.shape[1]))
            hist_urethra_3D = np.zeros((num_of_slices, 3*hist_urethra_crop.shape[0], 3*hist_urethra_crop.shape[1]))
            hist_cancer_3D = np.zeros((num_of_slices, 3*hist_cancer_crop.shape[0], 3*hist_cancer_crop.shape[1]))
            hist_landmark_3D = np.zeros((num_of_slices, 3*hist_landmark_crop.shape[0], 3*hist_landmark_crop.shape[1]))
            x_0 = x_m
            y_0 = y_m
            x_offset_0 = x_offset_m
            y_offset_0 = y_offset_m
            shift_x = hist_vol_crop.shape[0]
            shift_y = hist_vol_crop.shape[1]


        start_x = shift_x + int((x_m - x_offset_m - x_0 + x_offset_0)*spacing_mri/spacing_hist)
        start_y = shift_y + int((y_m - y_offset_m - y_0 + y_offset_0)*spacing_mri/spacing_hist)

        hist_vol_3D[s, start_x : hist_vol_crop.shape[0] + start_x,start_y:hist_vol_crop.shape[1] + start_y,:] =  cv2.cvtColor(hist_vol_crop, cv2.COLOR_RGB2BGR)  
        hist_mask_3D[s,start_x : hist_vol_crop.shape[0] + start_x,start_y:hist_vol_crop.shape[1] + start_y] =  hist_mask_crop[:,:,0] >0  
        hist_urethra_3D[s,start_x : hist_vol_crop.shape[0] + start_x,start_y:hist_vol_crop.shape[1] + start_y] =  hist_urethra_crop[:,:,0] >0   
        hist_cancer_3D[s,start_x : hist_vol_crop.shape[0] + start_x,start_y:hist_vol_crop.shape[1] + start_y] =  hist_cancer_crop[:,:,0]   >0 
        hist_landmark_3D[s,start_x : hist_vol_crop.shape[0] + start_x,start_y:hist_vol_crop.shape[1] + start_y] =  hist_landmark_crop[:,:,0] >0 
    
        
    hist_cancer_3D = sitk.GetImageFromArray(hist_cancer_3D*hist_mask_3D)    
    hist_vol_3D = sitk.GetImageFromArray(hist_vol_3D, isVector=True)
    hist_mask_3D = sitk.GetImageFromArray(hist_mask_3D)
    hist_urethra_3D = sitk.GetImageFromArray(hist_urethra_3D)
    hist_landmark_3D = sitk.GetImageFromArray(hist_landmark_3D)

    ### set physical info
    hist_vol_3D.SetSpacing((spacing_hist,spacing_hist,t2_vol.GetSpacing()[2]))
    hist_mask_3D.SetSpacing((spacing_hist,spacing_hist,t2_vol.GetSpacing()[2]))
    hist_cancer_3D.SetSpacing((spacing_hist,spacing_hist,t2_vol.GetSpacing()[2]))
    hist_urethra_3D.SetSpacing((spacing_hist,spacing_hist,t2_vol.GetSpacing()[2]))
    hist_landmark_3D.SetSpacing((spacing_hist,spacing_hist,t2_vol.GetSpacing()[2]))

    hist_vol_3D.SetDirection(t2_vol.GetDirection())
    hist_mask_3D.SetDirection(t2_vol.GetDirection())
    hist_cancer_3D.SetDirection(t2_vol.GetDirection())
    hist_urethra_3D.SetDirection(t2_vol.GetDirection())
    hist_landmark_3D.SetDirection(t2_vol.GetDirection())


    origin = t2_vol.TransformContinuousIndexToPhysicalPoint([ y_0 - y_offset_0 - shift_y*spacing_hist/spacing_mri, x_0 - x_offset_0 - shift_x*spacing_hist/spacing_mri, ind])

    hist_vol_3D.SetOrigin(origin)
    hist_mask_3D.SetOrigin(origin)
    hist_cancer_3D.SetOrigin(origin)
    hist_urethra_3D.SetOrigin(origin)
    hist_landmark_3D.SetOrigin(origin)   

    my_path = out_dir + '/'
    if not os.path.isdir(my_path):
        os.mkdir(path)

    sitk.WriteImage(hist_vol_3D,my_path + sub + '_moved_highres_rgb.nii.gz')
    sitk.WriteImage(hist_mask_3D,my_path + sub + '_moved_highres_region00_label.nii.gz')
    sitk.WriteImage(hist_cancer_3D,my_path + sub + '_moved_highres_region01_label.nii.gz')
    sitk.WriteImage(hist_urethra_3D,my_path + sub + '_moved_highres_region09_label.nii.gz')
    sitk.WriteImage(hist_landmark_3D,my_path + sub + '_moved_highres_region10_label.nii.gz')

    sitk.WriteImage(t2_ref,my_path + sub + '_fixed_image.nii.gz')

    t2_mask_img = sitk.GetImageFromArray(t2_mask_array)
    t2_mask_img.SetOrigin(t2_vol.GetOrigin())
    t2_mask_img.SetDirection(t2_vol.GetDirection())
    t2_mask_img.SetSpacing(t2_vol.GetSpacing())

    sitk.WriteImage(t2_mask_img>0,my_path + sub + '_fixed_mask_label.nii.gz')
    
    index = index + 1
    
    print("finish the registration of subject: " + sub)